# Fannie Mae analysis

(just started)

This notebook contains some python code to analyse mortgage data
See the following link how to download data and [more details](https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html).

Per quarter there is a Acquisition data set and a Performance data set. See the [details here](https://www.fanniemae.com/resources/file/fundmarket/pdf/webinar-101.pdf)


In [3]:
#### using the datatable package from h2o....  super package!
import pandas as pd
import numpy as np
import datatable as dt

## Import 2018 acquisition and performance data

The data available on mortgages is per quarter of starting mortgages (from 2000 until 2019). It is a zip containing two text files, for example if we look in the 2018Q1.zip file we have:

* the file Acquisition_2018Q1.txt, it contains all mortgaes that started in Q1 2018, each row is one mortgage,
* the file Performance_2018Q1.txt, it contains the performance of the mortgages in the acquisition file. Multiple rows in this file correspond to one mortgage, For every mortgage we have the monthly performance of the mortgage, form its start until Dec of 2019.

To make things managable I have downloaded only the four zip files corresponding to 2018, unzipped them and concatenated the Acquisition text files to one larger text file and zipped it. I did the same with the performances.

`cat Acq*.txt > acquisition.txt & zip acquisition.zip acquisition.txt`

`cat Perf*.txt > performance.txt & zip performance.zip performance.txt`

I am using the `fread` function from the datatable package, it can import zipped CSV files without extracting them.

In [5]:
%%time
#### import Acquisition data
acquisitions_Variables = [
    "LOAN_ID", "ORIG_CHN", "Seller_Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE",
    "FRST_DTE", "OLTV", "OCLTV", "NUM_BO", "Debt_to_Income", "Borrower_Credit_Score", "FTHB_FLG", "PURPOSE", "PROPERTY_TYPE",
    "NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", "MI_PCT", "Product_Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"
]

acquisition = dt.fread(
    "data/acquisition.zip",
    sep = "|",
    header = None ,
    columns = acquisitions_Variables,\
)

acquisition = acquisition.to_pandas()
acquisition.shape

CPU times: user 9.3 s, sys: 998 ms, total: 10.3 s
Wall time: 3.96 s


(1625195, 25)

In [6]:
### five random five records
acquisition.sample(5)

,LOAN_ID,ORIG_CHN,Seller_Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,PROPERTY_TYPE,NUM_UNIT,OCC_STAT,STATE,ZIP_3,MI_PCT,Product_Type,CSCORE_C,MI_TYPE,RELOCATION_FLG
1167622,819598588084,C,U.S. BANK N.A.,4.750,200000,360,06/2018,08/2018,83,83.0,...,PU,1,P,TN,380,12.0,FRM,NaN,1.0,N
639007,601090051160,R,"WELLS FARGO BANK, N.A.",4.500,110000,360,04/2018,06/2018,75,75.0,...,CO,1,P,FL,327,NaN,FRM,NaN,NaN,N
136894,387623739446,B,OTHER,4.500,546000,360,12/2017,02/2018,70,70.0,...,SF,2,P,NY,114,NaN,FRM,NaN,NaN,N
1539297,788968275653,R,U.S. BANK N.A.,4.875,187000,360,09/2018,11/2018,80,80.0,...,SF,1,P,TX,784,NaN,FRM,NaN,NaN,N
536571,358621192050,R,FAIRWAY INDEPENDENT MORTGAGE CORPORATION,4.625,288000,360,05/2018,07/2018,80,80.0,...,PU,1,S,AZ,863,NaN,FRM,NaN,NaN,N


In [7]:
%%time
#### Import performance data
performance_Variables = [
    "LOAN_ID", "Monthly_Rpt_Prd", "Servicer_Name", "LAST_RT", "LAST_UPB", "Loan_Age", "Months_To_Legal_Mat",
    "Adj_Month_To_Mat", "Maturity_Date", "MSA", "Delq_Status", "MOD_FLAG", "Zero_Bal_Code", "ZB_DTE", "LPI_DTE",
    "Foreclosure_date","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS", "CE_PROCS", "RMW_PROCS",
    "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"
]

performance = dt.fread(
    "data/performance.zip",
    sep = "|",
    header = None ,
    columns = performance_Variables
)

#performance = performance.to_pandas()
performance.shape

CPU times: user 57.1 s, sys: 14.1 s, total: 1min 11s
Wall time: 25 s


(28552851, 31)

In [8]:
%%time
performance = performance.to_pandas()

CPU times: user 51.4 s, sys: 36.3 s, total: 1min 27s
Wall time: 1min 17s


In [9]:
#### first 5 records
performance.head(5)

,LOAN_ID,Monthly_Rpt_Prd,Servicer_Name,LAST_RT,LAST_UPB,Loan_Age,Months_To_Legal_Mat,Adj_Month_To_Mat,Maturity_Date,MSA,...,TAX_COST,NS_PROCS,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG
0,100001040173,02/01/2018,QUICKEN LOANS INC.,4.25,NaN,0,360,360.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
1,100001040173,03/01/2018,,4.25,NaN,1,359,359.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
2,100001040173,04/01/2018,,4.25,NaN,2,358,358.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
3,100001040173,05/01/2018,,4.25,NaN,3,357,357.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
4,100001040173,06/01/2018,,4.25,NaN,4,356,356.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N


## Start with a simple analysis

This will be the easiest in terms of data prep. Look only at mortgages starting in one specific quarter. For the performance we look at foreclosure or not. So if there is a date in the Foreclosure_date column then the mortgage defaulted otherwise the mortgae did not defaulted. This is the worst that can happen to a mortgage. Latwer we will look at a different performance, days past due where the mortgage does not neccesarily goes default.


In [10]:
%%time
foreclosures = (
    performance
    .query("Foreclosure_date != ''")
    .filter(["LOAN_ID", "Foreclosure_date"])
)


CPU times: user 718 ms, sys: 315 ms, total: 1.03 s
Wall time: 1.07 s


In [11]:
foreclosures

,LOAN_ID,Foreclosure_date
19600,101917807151,10/01/2019
142167,113708792510,05/01/2019
249385,124076558673,08/01/2019
618991,159787643799,12/01/2018
679978,165820326073,10/01/2019
...,...,...
28070852,909577942160,11/01/2019
28116099,918106303345,08/01/2019
28157520,925789776954,09/01/2019
28331073,958169517513,08/01/2019


In [12]:
%%time
mortgages = (
    acquisition
    .merge(
        foreclosures,
        how="left",
        left_on="LOAN_ID",
        right_on="LOAN_ID"
    )
    .filter([ 
        "LOAN_ID","ORIG_DTE","FRST_DTE", "Debt_to_Income", "Borrower_Credit_Score", "PURPOSE",
        "Monthly_Rpt_Prd", "Loan_Age", "Seller_Name", "ORIG_RT",	"ORIG_AMT",
        "Zero_Bal_Code", "Delq_Status", "ZB_DTE", "LPI_DTE", "Foreclosure_date"
    ])
)

mortgages = (
    mortgages
    .assign(target = mortgages.Foreclosure_date.notna().astype(int))
)

CPU times: user 3.26 s, sys: 1.88 s, total: 5.14 s
Wall time: 5.65 s


In [13]:
mortgages

,LOAN_ID,ORIG_DTE,FRST_DTE,Debt_to_Income,Borrower_Credit_Score,PURPOSE,Seller_Name,ORIG_RT,ORIG_AMT,Foreclosure_date,target
0,100001040173,01/2018,03/2018,28.0,791.0,C,QUICKEN LOANS INC.,4.250,453000,NaN,0
1,100002370993,01/2018,03/2018,41.0,736.0,R,"WELLS FARGO BANK, N.A.",4.250,266000,NaN,0
2,100005405807,12/2017,01/2018,48.0,696.0,R,PMTT4,3.990,233000,NaN,0
3,100008071646,01/2018,03/2018,48.0,767.0,P,OTHER,4.250,184000,NaN,0
4,100010739040,02/2018,04/2018,22.0,727.0,R,OTHER,4.250,242000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
1625190,999988817135,09/2018,11/2018,49.0,723.0,C,OTHER,5.375,180000,NaN,0
1625191,999989388326,10/2018,12/2018,38.0,765.0,P,OTHER,4.625,257000,NaN,0
1625192,999991672082,07/2018,09/2018,17.0,725.0,C,"FLAGSTAR BANK, FSB",5.500,520000,NaN,0
1625193,999996626839,09/2018,11/2018,49.0,691.0,C,QUICKEN LOANS INC.,5.250,360000,NaN,0


In [14]:
mortgages.target.describe()

count    1.625195e+06
mean     1.975148e-04
std      1.405261e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: target, dtype: float64

In [15]:
1.975148e-04 *100

0.019751480000000002

So we see a default rate ofaround 0.518%.

We can also look at a different target. First time that a mortgage goes into 90 days or more past due. We can use the column `Delq_Status`, it is the Loan Delinquency status and has the following meaning:

* 0 - "Current or less than 30 days past due"
* 1 - "30 - 59 days past due"
* 2 - "60 - 89 days past due"
* 3 - "90 - 119 days past due"
* 4 - "120 - 149 days past due"
* 5 - "150 - 179 days past due"
* 6 - "180 Day Delinquency"
* 7 - "210 Day Delinquency"
* 8 - "240 Day Delinquency"
* 9 - "270 Day Delinquency" / "270+ Day Delinquency"

In [16]:
### select the status 3
tmp = (
    performance
    .query("Delq_Status == '3'")
    .filter(["LOAN_ID", "Monthly_Rpt_Prd", "Delq_Status"])
)

### select the first time when status 3 happened
tmp = (
    tmp
    .assign(date = pd.to_datetime(tmp.Monthly_Rpt_Prd))
    .sort_values(by = ["LOAN_ID", "Monthly_Rpt_Prd"])  
)

perf_90 = tmp.drop_duplicates(subset=["LOAN_ID"])

In [17]:
perf_90.shape

(8833, 4)

In [18]:
mortgages2 = (
    mortgages
    .merge(
        perf_90,
        how="left",
        left_on="LOAN_ID",
        right_on="LOAN_ID"
    )
)

mortgages2 = (
    mortgages2
    .assign(target_90 = mortgages2.date.notna().astype(int))
)

In [19]:
mortgages2

,LOAN_ID,ORIG_DTE,FRST_DTE,Debt_to_Income,Borrower_Credit_Score,PURPOSE,Seller_Name,ORIG_RT,ORIG_AMT,Foreclosure_date,target,Monthly_Rpt_Prd,Delq_Status,date,target_90
0,100001040173,01/2018,03/2018,28.0,791.0,C,QUICKEN LOANS INC.,4.250,453000,NaN,0,NaN,NaN,NaT,0
1,100002370993,01/2018,03/2018,41.0,736.0,R,"WELLS FARGO BANK, N.A.",4.250,266000,NaN,0,NaN,NaN,NaT,0
2,100005405807,12/2017,01/2018,48.0,696.0,R,PMTT4,3.990,233000,NaN,0,NaN,NaN,NaT,0
3,100008071646,01/2018,03/2018,48.0,767.0,P,OTHER,4.250,184000,NaN,0,NaN,NaN,NaT,0
4,100010739040,02/2018,04/2018,22.0,727.0,R,OTHER,4.250,242000,NaN,0,NaN,NaN,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625190,999988817135,09/2018,11/2018,49.0,723.0,C,OTHER,5.375,180000,NaN,0,NaN,NaN,NaT,0
1625191,999989388326,10/2018,12/2018,38.0,765.0,P,OTHER,4.625,257000,NaN,0,NaN,NaN,NaT,0
1625192,999991672082,07/2018,09/2018,17.0,725.0,C,"FLAGSTAR BANK, FSB",5.500,520000,NaN,0,NaN,NaN,NaT,0
1625193,999996626839,09/2018,11/2018,49.0,691.0,C,QUICKEN LOANS INC.,5.250,360000,NaN,0,NaN,NaN,NaT,0


In [20]:
mortgages2.filter(["target", "target_90"]).describe()

,target,target_90
count,1.625195e+06,1.625195e+06
mean,1.975148e-04,5.435040e-03
std,1.405261e-02,7.352213e-02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00


In [21]:
5.435040e-03 * 100

0.543504